In [12]:
import pandas as pd


# Load the dataset
file_path = 'training.1600000.processed.noemoticon.csv'
column_names = ['sentiment', 'id', 'date', 'query', 'user', 'text']
try:
    df = pd.read_csv(file_path, header=None, encoding='latin1', names=column_names)
except FileNotFoundError:
    print("Dataset file not found.")
    exit()
except Exception as e:
    print(f"Error loading dataset: {e}")
    exit()




/var/folders/c6/kh1v5smn18l3tbcsjpvvqv1h0000gn/T/ipykernel_19643/424644346.py:8: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, header=None, encoding='latin1', names=column_names)


In [13]:
# Convert text to lowercase
df['text_lower'] = df['text'].str.lower()
print("Sample DataFrame after converting to lowercase:")
print(df[['text', 'text_lower']].head())

Sample DataFrame after converting to lowercase:
                                                text  \
0                                 text of the tweet    
1  is upset that he can't update his Facebook by ...   
2  @Kenichan I dived many times for the ball. Man...   
3    my whole body feels itchy and like its on fire    
4  @nationwideclass no, it's not behaving at all....   

                                          text_lower  
0                                 text of the tweet   
1  is upset that he can't update his facebook by ...  
2  @kenichan i dived many times for the ball. man...  
3    my whole body feels itchy and like its on fire   
4  @nationwideclass no, it's not behaving at all....  


In [14]:
import string
# Remove punctuation
punctuation = string.punctuation

def remove_punctuation(text):
    translator = str.maketrans('', '', punctuation)
    return text.translate(translator)

In [15]:
df['text_without_punctuation'] = df['text_lower'].apply(remove_punctuation)
print("\nSample DataFrame after removing punctuation:")
print(df[['text_lower', 'text_without_punctuation']].head())


Sample DataFrame after removing punctuation:
                                          text_lower  \
0                                 text of the tweet    
1  is upset that he can't update his facebook by ...   
2  @kenichan i dived many times for the ball. man...   
3    my whole body feels itchy and like its on fire    
4  @nationwideclass no, it's not behaving at all....   

                            text_without_punctuation  
0                                 text of the tweet   
1  is upset that he cant update his facebook by t...  
2  kenichan i dived many times for the ball manag...  
3    my whole body feels itchy and like its on fire   
4  nationwideclass no its not behaving at all im ...  


In [22]:
import string
import nltk
from nltk.corpus import stopwords
# Download stopwords if not already downloaded
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')

In [23]:
# Get the set of English stop words
stop_words = set(stopwords.words('english'))
print(f"تعداد stop words انگلیسی: {len(stop_words)}")
print(f"نمونه‌ای از stop words: {list(stop_words)[:10]}")

تعداد stop words انگلیسی: 198
نمونه‌ای از stop words: ['do', "she's", "doesn't", 'y', 'in', "isn't", 'been', 'ma', 'he', 'because']


In [24]:
# Function to remove stop words from text (using a for loop)
def remove_stopwords_loop(text):
    words = text.split()
    filtered_words = []
    for word in words:
        if word.lower() not in stop_words:
            filtered_words.append(word)
    return " ".join(filtered_words)

In [27]:
df['text_without_stopwords'] = df['text_without_punctuation'].apply(remove_stopwords_loop)

In [28]:
print(df['text_without_stopwords'].head())

0                                           text tweet
1    upset cant update facebook texting might cry r...
2    kenichan dived many times ball managed save 50...
3                     whole body feels itchy like fire
4             nationwideclass behaving im mad cant see
Name: text_without_stopwords, dtype: object


In [29]:
print(df['text_without_stopwords'])

0                                                 text tweet
1          upset cant update facebook texting might cry r...
2          kenichan dived many times ball managed save 50...
3                           whole body feels itchy like fire
4                   nationwideclass behaving im mad cant see
                                 ...                        
1048568                            grandma making dinenr mum
1048569        midmorning snack time bowl cheese noodles yum
1048570    shadela say like terminiator movies comes like...
1048571                  destinyhope92 im great thaanks wbuu
1048572                           cant wait til date weekend
Name: text_without_stopwords, Length: 1048573, dtype: object


In [30]:
df.to_csv('your_processed_data.csv', index=False)